In [ ]:
import glob
import optuna
import joblib
import itertools
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import defaultdict
from scipy import stats
from scipy.stats import wilcoxon, mannwhitneyu

In [ ]:
optuna.__version__

# BRN Hyper-Parameter Importances

In [ ]:
studies = glob.glob('path/to/studiesBRRN-NODA*.pkl')
results = defaultdict(list)
for study_path in studies:
    study_keys = [
             'learning_rate',
             'weight_init',
             'ucb_scale',
             'increase_prune_plateau_count_plateau_count',
             'perform_prune_quantile',
             'n_layers',
             'delta',
             'add_negations',
             'layer_sizes',
             'early_stopping_plateau_count',
             'T_0',
             'perform_prune_plateau_count',
             'T_mult',
             'prune_strategy',
             'increase_prune_plateau_count',
             'use_lookahead',
             'normal_form',
             'use_weight_decay',
             'swa',
             'bootstrap',
             # 'use_l1',
             # 'use_early_stopping',
             # 'augment',
             'n_selected_features_input',
             'n_selected_features_output',
             'n_selected_features_internal']
    study = joblib.load(study_path)
    ie = optuna.importance.FanovaImportanceEvaluator()
    imp = ie.evaluate(study)
    for k in study_keys:
        if k in imp:
            results[k] += [imp[k]]
        else:
            results[k] += [0]

In [ ]:
columns = list(results.keys())
means = [np.mean(v) for v in results.values()]
stdevs = [np.std(v) for v in results.values()]

In [ ]:
df = pd.DataFrame({'features': columns, 'means': means, 'stdevs': stdevs})
df.sort_values('means', ascending=True, inplace=True)

In [ ]:
plt.barh(
    list(df['features']), 
    list(df['means']), 
    # xerr=list((df['stdevs'] * 1.96).tolist()), 
    capsize=6, color='tab:blue'
)

plt.ylabel("Hyper-Parameter")
plt.xlabel("Importance")
plt.title("Mean Hyper-Parameter Importances")

plt.grid()
plt.tight_layout()
plt.savefig("path/to/figures/parameter_importance.png")
plt.show()

# Compute Time

In [ ]:
studies = glob.glob('/path/to/studies/NAM-HO*.pkl')

In [ ]:
study = joblib.load(studies[0])

In [ ]:
study_df = study.trials_dataframe()

In [ ]:
study_df[study_df['state']=='COMPLETE'].shape

In [ ]:
study_df['duration'].iloc[0]

In [ ]:
def collect_compute_time_by_model(studies):
    results = defaultdict(list)
    best_trials = defaultdict(list)
    for i, study_path in enumerate(studies):
        study = joblib.load(study_path)
        study_df = study.trials_dataframe()
        if study_df[study_df['state'] == 'COMPLETE'].shape[0] == 400:
            print(study_df[study_df['state'] == 'COMPLETE'].shape)
            study_df = study_df[study_df['state'] == 'COMPLETE']    
        else:
            print(f"did not find 400 complete trials {study_df[study_df['state'] == 'COMPLETE'].shape[0]}")
            study_df = study_df[study_df['state'] != 'FAIL']
            study_df = study_df.head(400)
            print(f"using {study_df.shape[0]} non failed trials")
        results[str(i) + '_study_duration'] = study_df['duration'].values.tolist()
        if len(results[str(i) + '_study_duration']) < 400:
            results[str(i) + '_study_duration'] += [np.nan] * (400 - len(results[str(i) + '_study_duration']))
    results = pd.DataFrame(results)
    return results

In [ ]:
studies = glob.glob('path/to/studies/BRRN-NODA*.pkl')
brn_durations = collect_compute_time_by_model(studies)

In [ ]:
studies = glob.glob('path/to/studies/RF*.pkl')
rf_durations = collect_compute_time_by_model(studies)

In [ ]:
studies = glob.glob('path/to/studies/GBT*.pkl')
gbt_durations = collect_compute_time_by_model(studies)

In [ ]:
studies = glob.glob('path/to/studies/XGB*.pkl')
xgb_durations = collect_compute_time_by_model(studies)

In [ ]:
studies = glob.glob('path/to/studies/PYTORCH-NN-NODA-200*.pkl')
mlp_durations = collect_compute_time_by_model(studies)

In [ ]:
studies = glob.glob('path/to/studies/FTT-NODA*.pkl')
ftt_durations = collect_compute_time_by_model(studies)

In [ ]:
studies = glob.glob('path/to/studies/NAM-HO*.pkl')
nam_durations = collect_compute_time_by_model(studies)

In [ ]:
studies = glob.glob('path/to/studies/BRCG*.pkl')
brcg_durations = collect_compute_time_by_model(studies)

In [ ]:
studies = glob.glob('path/to/studies/DANet-200*.pkl')
dan_durations = collect_compute_time_by_model(studies)

In [ ]:
studies = glob.glob('path/to/studies/Cofrnet*.pkl')
cof_durations = collect_compute_time_by_model(studies)

In [ ]:
studies = glob.glob('path/to/studies/Difflogic*.pkl')
dif_durations = collect_compute_time_by_model(studies)

In [ ]:
brn_durations.applymap(lambda x: pd.Timedelta(x)).mean().mean()

In [ ]:
rf_durations.applymap(lambda x: pd.Timedelta(x)).mean().mean()

In [ ]:
gbt_durations.applymap(lambda x: pd.Timedelta(x)).mean().mean()

In [ ]:
xgb_durations.applymap(lambda x: pd.Timedelta(x)).mean().mean()

In [ ]:
mlp_durations.applymap(lambda x: pd.Timedelta(x)).mean().mean()

In [ ]:
ftt_durations.applymap(lambda x: pd.Timedelta(x)).mean(skipna=True).mean(skipna=True)

In [ ]:
nam_durations.applymap(lambda x: pd.Timedelta(x)).mean(skipna=True).mean(skipna=True)

In [ ]:
brcg_durations.applymap(lambda x: pd.Timedelta(x)).mean(skipna=True).mean(skipna=True)

In [ ]:
dan_durations.applymap(lambda x: pd.Timedelta(x)).mean(skipna=True).mean(skipna=True)

In [ ]:
cof_durations.applymap(lambda x: pd.Timedelta(x)).mean(skipna=True).mean(skipna=True)

In [ ]:
dif_durations.applymap(lambda x: pd.Timedelta(x)).mean(skipna=True).mean(skipna=True)

# BRN Use of FeatureBinarization

In [ ]:
studies = glob.glob('path/to/studies/BRRN-NODA*.pkl')
use_fbt = []
for study_path in studies:
    study = joblib.load(study_path)
    best_params = study.best_params
    if 'use_fbt' in best_params:
        use_fbt += [best_params['use_fbt']]
    else:
        use_fbt += [False]   

In [ ]:
sum(use_fbt), len(use_fbt), sum(use_fbt)/len(use_fbt)

# MLP Use of FeatureBinarization

In [ ]:
studies = glob.glob('path/to/studies/PYTORCH-NN-NODA-200*.pkl')
use_fbt = []
for study_path in studies:
    study = joblib.load(study_path)
    best_params = study.best_params
    if 'use_fbt' in best_params:
        use_fbt += [best_params['use_fbt']]
    else:
        use_fbt += [False]  

In [ ]:
sum(use_fbt), len(use_fbt), sum(use_fbt)/len(use_fbt)

# Optimization History

In [ ]:
studies = glob.glob('path/to/studies/BRRN-NODA*.pkl')
study = joblib.load(studies[0])
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

In [ ]:
study_df = study.trials_dataframe()

In [ ]:
study_df.iloc[study_df['value'].argmax()]['number']

In [ ]:
study_df = study_df[study_df['state'] == 'COMPLETE']

In [ ]:
study_df = study_df.head(400)

In [ ]:
study_df['value'].values.tolist()

In [ ]:
def collect_optimization_histories_by_model(studies, model_name='BRN'):
    results = defaultdict(list)
    best_trials = defaultdict(list)
    for i, study_path in enumerate(studies):
        study = joblib.load(study_path)
        study_df = study.trials_dataframe()
        if study_df[study_df['state'] == 'COMPLETE'].shape[0] == 400:
            print(study_df[study_df['state'] == 'COMPLETE'].shape)
            study_df = study_df[study_df['state'] == 'COMPLETE']    
        else:
            print(f"did not find 400 complete trials {study_df[study_df['state'] == 'COMPLETE'].shape[0]}")
            study_df = study_df[study_df['state'] != 'FAIL']
            study_df = study_df.head(400)
            print(f"using {study_df.shape[0]} non failed trials")
        results[str(i) + '_study_value'] = study_df['value'].values.tolist()
        if len(results[str(i) + '_study_value']) < 400:
            results[str(i) + '_study_value'] += [np.nan] * (400 - len(results[str(i) + '_study_value']))
        best_trials[str(i) + '_study_best_trial'] = [study_df.iloc[study_df['value'].argmax()]['number']]
    results = pd.DataFrame(results)
    best_trials = pd.DataFrame(best_trials)
    return results, best_trials

In [ ]:
studies = glob.glob('path/to/studies/BRRN-NODA*.pkl')
brn_optimization_histories, brn_best_trials = collect_optimization_histories_by_model(studies, model_name='BRN')
normalized_brn_optimization_histories = brn_optimization_histories/brn_optimization_histories.iloc[0, :]
brn_optimization_histories_mean = normalized_brn_optimization_histories.mean(axis=1)
brn_optimization_histories_std = normalized_brn_optimization_histories.std(axis=1)
brn_optimization_histories_ci = 1.96 * brn_optimization_histories_std / np.sqrt(brn_optimization_histories.shape[1])

In [ ]:
studies = glob.glob('path/to/studies/RF*.pkl')
rf_optimization_histories, rf_best_trials = collect_optimization_histories_by_model(studies, model_name='RF')
normalized_rf_optimization_histories = rf_optimization_histories/brn_optimization_histories.iloc[0, :]
rf_optimization_histories_mean = normalized_rf_optimization_histories.mean(axis=1)
rf_optimization_histories_std = normalized_rf_optimization_histories.std(axis=1)
rf_optimization_histories_ci = 1.96 * rf_optimization_histories_std / np.sqrt(rf_optimization_histories.shape[1])

In [ ]:
studies = glob.glob('path/to/studies/GBT*.pkl')
gbt_optimization_histories, gbt_best_trials = collect_optimization_histories_by_model(studies, model_name='GBT')
normalized_gbt_optimization_histories = gbt_optimization_histories/brn_optimization_histories.iloc[0, :]
gbt_optimization_histories_mean = normalized_gbt_optimization_histories.mean(axis=1)
gbt_optimization_histories_std = normalized_gbt_optimization_histories.std(axis=1)
gbt_optimization_histories_ci = 1.96 * gbt_optimization_histories_std / np.sqrt(gbt_optimization_histories.shape[1])

In [ ]:
studies = glob.glob('path/to/studies/XGB*.pkl')
xgb_optimization_histories, xgb_best_trials = collect_optimization_histories_by_model(studies, model_name='XGB')
normalized_xgb_optimization_histories = xgb_optimization_histories/brn_optimization_histories.iloc[0, :]
xgb_optimization_histories_mean = normalized_xgb_optimization_histories.mean(axis=1)
xgb_optimization_histories_std = normalized_xgb_optimization_histories.std(axis=1)
xgb_optimization_histories_ci = 1.96 * xgb_optimization_histories_std / np.sqrt(xgb_optimization_histories.shape[1])

In [ ]:
studies = glob.glob('path/to/studies/PYTORCH-NN-NODA-200*.pkl')
mlp_optimization_histories, mlp_best_trials = collect_optimization_histories_by_model(studies, model_name='MLP')
normalized_mlp_optimization_histories = mlp_optimization_histories/brn_optimization_histories.iloc[0, :]
mlp_optimization_histories_mean = normalized_mlp_optimization_histories.mean(axis=1)
mlp_optimization_histories_std = normalized_mlp_optimization_histories.std(axis=1)
mlp_optimization_histories_ci = 1.96 * mlp_optimization_histories_std / np.sqrt(mlp_optimization_histories.shape[1])

In [ ]:
studies = glob.glob('path/to/studies/FTT-NODA*.pkl')
ftt_optimization_histories, ftt_best_trials = collect_optimization_histories_by_model(studies, model_name='FTT')
normalized_ftt_optimization_histories = ftt_optimization_histories/brn_optimization_histories.iloc[0, :]
ftt_optimization_histories_mean = normalized_ftt_optimization_histories.mean(axis=1)
ftt_optimization_histories_std = normalized_ftt_optimization_histories.std(axis=1)
ftt_optimization_histories_ci = 1.96 * ftt_optimization_histories_std / np.sqrt(ftt_optimization_histories.shape[1])

In [ ]:
studies = glob.glob('path/to/studies/DANet-200*.pkl')
dan_optimization_histories, dan_best_trials = collect_optimization_histories_by_model(studies, model_name='DAN')
normalized_dan_optimization_histories = dan_optimization_histories/brn_optimization_histories.iloc[0, :]
dan_optimization_histories_mean = normalized_dan_optimization_histories.mean(axis=1)
dan_optimization_histories_std = normalized_dan_optimization_histories.std(axis=1)
dan_optimization_histories_ci = 1.96 * dan_optimization_histories_std / np.sqrt(dan_optimization_histories.shape[1])

In [ ]:
studies = glob.glob('path/to/studies/NAM-HO*.pkl')
nam_optimization_histories, nam_best_trials = collect_optimization_histories_by_model(studies, model_name='NAM')
normalized_nam_optimization_histories = nam_optimization_histories/brn_optimization_histories.iloc[0, :]
nam_optimization_histories_mean = normalized_nam_optimization_histories.mean(axis=1)
nam_optimization_histories_std = normalized_nam_optimization_histories.std(axis=1)
nam_optimization_histories_ci = 1.96 * nam_optimization_histories_std / np.sqrt(nam_optimization_histories.shape[1])

In [ ]:
studies = glob.glob('path/to/studies/Cofrnet*.pkl')
cfn_optimization_histories, cfn_best_trials = collect_optimization_histories_by_model(studies, model_name='CFN')
normalized_cfn_optimization_histories = cfn_optimization_histories/brn_optimization_histories.iloc[0, :]
cfn_optimization_histories_mean = normalized_cfn_optimization_histories.mean(axis=1)
cfn_optimization_histories_std = normalized_cfn_optimization_histories.std(axis=1)
cfn_optimization_histories_ci = 1.96 * cfn_optimization_histories_std / np.sqrt(cfn_optimization_histories.shape[1])

In [ ]:
studies = glob.glob('path/to/studies/BRCG*.pkl')
brcg_optimization_histories, brcg_best_trials = collect_optimization_histories_by_model(studies, model_name='BRCG')
normalized_brcg_optimization_histories = brcg_optimization_histories/brn_optimization_histories.iloc[0, :]
brcg_optimization_histories_mean = normalized_brcg_optimization_histories.mean(axis=1)
brcg_optimization_histories_std = normalized_brcg_optimization_histories.std(axis=1)
brcg_optimization_histories_ci = 1.96 * brcg_optimization_histories_std / np.sqrt(brcg_optimization_histories.shape[1])

In [ ]:
studies = glob.glob('path/to/studies/Difflogic*.pkl')
dif_optimization_histories, dif_best_trials = collect_optimization_histories_by_model(studies, model_name='DIF')
normalized_dif_optimization_histories = dif_optimization_histories/brn_optimization_histories.iloc[0, :]
dif_optimization_histories_mean = normalized_dif_optimization_histories.mean(axis=1)
dif_optimization_histories_std = normalized_dif_optimization_histories.std(axis=1)
dif_optimization_histories_ci = 1.96 * dif_optimization_histories_std / np.sqrt(dif_optimization_histories.shape[1])

In [ ]:
#some example data
x = np.arange(0, 400)
y = brn_optimization_histories_mean.rolling(5).mean()
y1 = rf_optimization_histories_mean.rolling(5).mean()
y2 = gbt_optimization_histories_mean.rolling(5).mean()
y3 = xgb_optimization_histories_mean.rolling(5).mean()
y4 = mlp_optimization_histories_mean.rolling(5).mean()
y5 = ftt_optimization_histories_mean.rolling(5).mean()
y6 = dan_optimization_histories_mean.rolling(5).mean()
y7 = nam_optimization_histories_mean.rolling(5).mean()
y8 = cfn_optimization_histories_mean.rolling(5).mean()
y9 = brcg_optimization_histories_mean.rolling(5).mean()
y10 = dif_optimization_histories_mean.rolling(5).mean()
#some confidence interval
ci = brn_optimization_histories_ci.rolling(5).mean()
ci1 = rf_optimization_histories_ci.rolling(5).mean()
ci2 = gbt_optimization_histories_ci.rolling(5).mean()
ci3 = xgb_optimization_histories_ci.rolling(5).mean()
ci4 = mlp_optimization_histories_ci.rolling(5).mean()
ci5 = ftt_optimization_histories_ci.rolling(5).mean()
ci6 = dan_optimization_histories_ci.rolling(5).mean()
ci7 = nam_optimization_histories_ci.rolling(5).mean()
ci8 = cfn_optimization_histories_ci.rolling(5).mean()
ci9 = brcg_optimization_histories_ci.rolling(5).mean()
ci10 = dif_optimization_histories_ci.rolling(5).mean()

plt.style.use('seaborn-v0_8-paper')

fig, ax = plt.subplots()

ax.plot(x,y, label='BRN', linewidth=0.9)
ax.plot(x,y1, label='RF', linewidth=0.9)
ax.plot(x,y2, label='GBT', linewidth=0.9)
ax.plot(x,y3, label='XGB', linewidth=0.9)
ax.plot(x,y4, label='MLP', linewidth=0.9)
ax.plot(x,y5, label='FTT', linewidth=0.9)
ax.plot(x,y6, label='DAN', linewidth=0.9)
ax.plot(x,y7, label='NAM', linewidth=0.9)
ax.plot(x,y8, label='CFN', linewidth=0.9)
ax.plot(x,y9, label='BRCG', linewidth=0.9)
ax.plot(x,y10, label='DIF', linewidth=0.9)

ax.fill_between(x, (y-ci), (y+ci), alpha=.1)
ax.fill_between(x, (y1-ci1), (y1+ci1), alpha=.1)
ax.fill_between(x, (y2-ci2), (y2+ci2), alpha=.1)
ax.fill_between(x, (y3-ci3), (y3+ci3), alpha=.1)
ax.fill_between(x, (y4-ci4), (y4+ci4), alpha=.1)
ax.fill_between(x, (y5-ci5), (y5+ci5), alpha=.1)
ax.fill_between(x, (y6-ci6), (y6+ci6), alpha=.1)
ax.fill_between(x, (y7-ci7), (y7+ci7), alpha=.1)
ax.fill_between(x, (y8-ci8), (y8+ci8), alpha=.1)
ax.fill_between(x, (y9-ci9), (y9+ci9), alpha=.1)
ax.fill_between(x, (y10-ci10), (y10+ci10), alpha=.1)

# BRN MAX

# ymax = max(y)
# xpos = np.argmax(y)
# xmax = x[xpos]
# ax.annotate('BRN Max', xy=(xmax, ymax-0.001), xytext=(xmax, ymax-0.1),
#             arrowprops=dict(facecolor='black', headwidth=5, width=1, headlength=4),)

brn_avg_max = np.ceil(brn_best_trials.mean(axis=1).values[0]).astype('int')
ax.axvline(brn_avg_max, color='C0', linewidth=0.9)


# # RF MAX
# ymax = max(y1)
# xpos = np.argmax(y1)
# xmax = x[xpos]
# ax.annotate('RF Max', xy=(xmax, ymax), xytext=(xmax, ymax+0.05),
#             arrowprops=dict(facecolor='black', headwidth=5, width=1, headlength=4),)

rf_avg_max = np.ceil(rf_best_trials.mean(axis=1).values[0]).astype('int')
print(rf_avg_max)
ax.axvline(rf_avg_max, color='C1', linewidth=1)

# # GBT MAX
# ymax = max(y2)
# xpos = np.argmax(y2)
# xmax = x[xpos]
# ax.annotate('GBT Max', xy=(xmax, ymax), xytext=(xmax, ymax+0.05),
#             arrowprops=dict(facecolor='black', headwidth=5, width=1, headlength=4),)

gbt_avg_max = np.ceil(gbt_best_trials.mean(axis=1).values[0]).astype('int')
print(gbt_avg_max)
ax.axvline(gbt_avg_max, color='C2', linewidth=0.9)

# # XGB MAX
# ymax = max(y3)
# xpos = np.argmax(y3)
# xmax = x[xpos]
# ax.annotate('XGB Max', xy=(xmax, ymax), xytext=(xmax, ymax+0.05),
#             arrowprops=dict(facecolor='black', headwidth=5, width=1, headlength=4),)

xgb_avg_max = np.ceil(xgb_best_trials.mean(axis=1).values[0]).astype('int')
ax.axvline(xgb_avg_max, color='C3', linewidth=0.9)

# # MLP MAX
# ymax = max(y3)
# xpos = np.argmax(y3)
# xmax = x[xpos]
# ax.annotate('XGB Max', xy=(xmax, ymax), xytext=(xmax, ymax+0.05),
#             arrowprops=dict(facecolor='black', headwidth=5, width=1, headlength=4),)

mlp_avg_max = np.ceil(mlp_best_trials.mean(axis=1).values[0]).astype('int')
ax.axvline(mlp_avg_max, color='C4', linewidth=0.9)

# # FTT MAX
# ymax = max(y3)
# xpos = np.argmax(y3)
# xmax = x[xpos]
# ax.annotate('XGB Max', xy=(xmax, ymax), xytext=(xmax, ymax+0.05),
#             arrowprops=dict(facecolor='black', headwidth=5, width=1, headlength=4),)

ftt_avg_max = np.ceil(ftt_best_trials.mean(axis=1).values[0]).astype('int')
ax.axvline(ftt_avg_max, color='C5', linewidth=0.9)

dan_avg_max = np.ceil(dan_best_trials.mean(axis=1).values[0]).astype('int')
ax.axvline(dan_avg_max, color='C6', linewidth=0.9)

nam_avg_max = np.ceil(nam_best_trials.mean(axis=1).values[0]).astype('int')
ax.axvline(nam_avg_max, color='C7', linewidth=0.9)

cfn_avg_max = np.ceil(cfn_best_trials.mean(axis=1).values[0]).astype('int')
ax.axvline(cfn_avg_max, color='C8', linewidth=0.9)

brcg_avg_max = np.ceil(brcg_best_trials.mean(axis=1).values[0]).astype('int')
ax.axvline(brcg_avg_max, color='C9', linewidth=0.9)

dif_avg_max = np.ceil(dif_best_trials.mean(axis=1).values[0]).astype('int')
ax.axvline(dif_avg_max, color='C10', linewidth=0.9)

ax.legend(loc='lower right')
plt.title("Optimization Histories w/ 95% Confidence Interval")
plt.xlabel("Trial Number")
plt.ylabel("Average Normalized Validation AUC")
plt.savefig("path/to/plots/optimization_history.png")

In [ ]:
print plt.style.available

# Results Box Plot

In [ ]:
df = pd.read_csv('path/to/results/results_reproducible_raw_data.csv')

In [ ]:
df_box = pd.DataFrame(
    {
        'BRN': df[['BRN NODA 1', 'BRN NODA 2', 'BRN NODA 3', 'BRN NODA 4', 'BRN NODA 5']].mean(axis=1),
        'BRCG': df[['BRCG 1', 'BRCG 2', 'BRCG 3', 'BRCG 4', 'BRCG 5']].mean(axis=1),
        'DIF': df[['Dif 1', 'Dif 2', 'Dif 3', 'Dif 4', 'Dif 5']].mean(axis=1),
        'NAM': df[['NAM 1', 'NAM 2', 'NAM 3', 'NAM 4', 'NAM 5']].mean(axis=1),
        'CFN': df[['CoFR 1', 'CoFR 2', 'CoFR 3', 'CoFR 4', 'CoFR 5']].mean(axis=1),
        'FTT': df[['FFT 1', 'FFT 2', 'FFT 3', 'FFT 4', 'FFT 5']].mean(axis=1),
        'DAN': df[['DANet 1', 'DANet 2', 'DANet 3', 'DANet 4', 'DANet 5']].mean(axis=1),
        'MLP': df[['MLP NODA 1', 'MLP NODA 2', 'MLP NODA 3', 'MLP NODA 4', 'MLP NODA 5']].mean(axis=1),
        'RF': df[['RF 1', 'RF 2', 'RF 3', 'RF 4', 'RF 5']].mean(axis=1),
        'XGB': df[['XGB 1', 'XGB 2', 'XGB 3', 'XGB 4', 'XGB 5']].mean(axis=1),
        'GBT': df[['GBT 1', 'GBT 2', 'GBT 3', 'GBT 4', 'GBT 5']].mean(axis=1),
    }
)

In [ ]:
plt.style.use('seaborn-v0_8-paper')
fig, ax = plt.subplots()
ax = df_box.plot.box(ax=ax, title="Test AUC Distributions", ylabel="AUC")
plt.savefig("path/to/plots/auc_distributions.png")

In [ ]:
df_confidence = pd.DataFrame(
    {
        'BRN': df[['BRN NODA 1', 'BRN NODA 2', 'BRN NODA 3', 'BRN NODA 4', 'BRN NODA 5']].mean(axis=1),
        'BRCG': df[['BRCG 1', 'BRCG 2', 'BRCG 3', 'BRCG 4', 'BRCG 5']].mean(axis=1),
        'DIF': df[['Dif 1', 'Dif 2', 'Dif 3', 'Dif 4', 'Dif 5']].mean(axis=1),
        'NAM': df[['NAM 1', 'NAM 2', 'NAM 3', 'NAM 4', 'NAM 5']].mean(axis=1),
        'CFN': df[['CoFR 1', 'CoFR 2', 'CoFR 3', 'CoFR 4', 'CoFR 5']].mean(axis=1),
        'FTT': df[['FFT 1', 'FFT 2', 'FFT 3', 'FFT 4', 'FFT 5']].mean(axis=1),
        'DAN': df[['DANet 1', 'DANet 2', 'DANet 3', 'DANet 4', 'DANet 5']].mean(axis=1),
        'MLP': df[['MLP NODA 1', 'MLP NODA 2', 'MLP NODA 3', 'MLP NODA 4', 'MLP NODA 5']].mean(axis=1),
        'RF': df[['RF 1', 'RF 2', 'RF 3', 'RF 4', 'RF 5']].mean(axis=1),
        'XGB': df[['XGB 1', 'XGB 2', 'XGB 3', 'XGB 4', 'XGB 5']].mean(axis=1),
        'GBT': df[['GBT 1', 'GBT 2', 'GBT 3', 'GBT 4', 'GBT 5']].mean(axis=1),
        'BRN_CI': df[['BRN NODA 1', 'BRN NODA 2', 'BRN NODA 3', 'BRN NODA 4', 'BRN NODA 5']].std(axis=1) * 1.96 / np.sqrt(5),
        'BRCG_CI': df[['BRCG 1', 'BRCG 2', 'BRCG 3', 'BRCG 4', 'BRCG 5']].std(axis=1) * 1.96 / np.sqrt(5),
        'DIF_CI': df[['Dif 1', 'Dif 2', 'Dif 3', 'Dif 4', 'Dif 5']].std(axis=1) * 1.96 / np.sqrt(5),
        'NAM_CI': df[['NAM 1', 'NAM 2', 'NAM 3', 'NAM 4', 'NAM 5']].std(axis=1) * 1.96 / np.sqrt(5),
        'CFN_CI': df[['CoFR 1', 'CoFR 2', 'CoFR 3', 'CoFR 4', 'CoFR 5']].std(axis=1) * 1.96 / np.sqrt(5),
        'FTT_CI': df[['FFT 1', 'FFT 2', 'FFT 3', 'FFT 4', 'FFT 5']].std(axis=1) * 1.96 / np.sqrt(5),
        'DAN_CI': df[['DANet 1', 'DANet 2', 'DANet 3', 'DANet 4', 'DANet 5']].std(axis=1) * 1.96 / np.sqrt(5),
        'MLP_CI': df[['MLP NODA 1', 'MLP NODA 2', 'MLP NODA 3', 'MLP NODA 4', 'MLP NODA 5']].std(axis=1) * 1.96 / np.sqrt(5),
        'RF_CI': df[['RF 1', 'RF 2', 'RF 3', 'RF 4', 'RF 5']].std(axis=1) * 1.96 / np.sqrt(5),
        'XGB_CI': df[['XGB 1', 'XGB 2', 'XGB 3', 'XGB 4', 'XGB 5']].std(axis=1) * 1.96 / np.sqrt(5),
        'GBT_CI': df[['GBT 1', 'GBT 2', 'GBT 3', 'GBT 4', 'GBT 5']].std(axis=1) * 1.96 / np.sqrt(5),
    }
)
df_confidence = df_confidence.mean()

df_confidence = pd.DataFrame({
    'Model': df_confidence.iloc[:11].index,
    'Mean Test AUC': df_confidence.iloc[:11].values, 
    'ci': df_confidence.iloc[11:].values})


In [ ]:
df_confidence

In [ ]:
ax = df_confidence.plot(
    kind='scatter', 
    x='Model', 
    y='Mean Test AUC', 
    # yerr=dict(yerr='ci', ecolor='gray', lw=2, capsize=5, capthick=2), xlabel='', 
    grid=True, 
    yticks=np.arange(0.6, 0.90, 0.025),
    title="Mean Test AUC with 95% Confidence Intervals",
    s=40,
    xlabel=""
)
error_kwargs = dict(ecolor='black', lw=1, capsize=3, capthick=1, fmt='o')
plt.errorbar(x=df_confidence['Model'], y=df_confidence['Mean Test AUC'], yerr=df_confidence['ci'], **error_kwargs)
plt.savefig("path/to/plots/mean_auc_confidence.png")

# Statistical Test

In [ ]:
from scipy.stats import wilcoxon
import itertools

In [ ]:
df = pd.read_csv('path/to/results/results_reproducible_raw_data.csv')

### By Dataset

In [ ]:
a = set([x[:-2] for x in df.columns])
b = set([x[:-2] for x in df.columns])

algo_types = set([x[:-2] for x in df.columns])

dataset = []
alg1 = []
alg2 = []
pval = []
for i in range(22):
    df_stats_ds = df.iloc[i, :]
    for r in itertools.product(a, b):
        dataset += [i]
        alg1 += [r[0]]
        alg2 += [r[1]]
        if r[0] != r[1]:
            df_stats_r0 = df_stats_ds[[f"{r[0]} {j}" for j in range(1, 6)]].T.values
            df_stats_r1 = df_stats_ds[[f"{r[1]} {j}" for j in range(1, 6)]].T.values
            d = np.around(df_stats_r0 - df_stats_r1, decimals=6)
            try:
                # res = wilcoxon(d, alternative='greater')
                res = mannwhitneyu(df_stats_r0, df_stats_r1, alternative="greater")
            except:
                print(d, r)
            pval += [res.pvalue]
        else:
            pval += [np.nan]
    
results = pd.DataFrame({'dataset': dataset, 'alg1': alg1, 'alg2': alg2, 'pvalue_greater': pval})
results['significant'] = results['pvalue_greater'] <= 0.05
results = results[['dataset', 'alg1', 'alg2', 'significant']].set_index(['dataset', 'alg1', 'alg2']).unstack()['significant']

#### Produce Win/Loss/Tie

In [ ]:
for col in results.columns:
    compare_algo = col
    print(compare_algo, results[compare_algo].loc[:, 'BRN NODA'].sum(), "/", 
          results['BRN NODA'].loc[:, compare_algo].sum(), "/",
          22 - results[compare_algo].loc[:, 'BRN NODA'].sum() - results['BRN NODA'].loc[:, compare_algo].sum()
    )

#### Produce Statisticall Significant Best per Dataset

In [ ]:
results['beats_all_algos'] = results.apply(lambda x: sum(x) == 10, axis=1)

In [ ]:
results[results['beats_all_algos']]

### Overall Benchmark Method 2

In [ ]:
df_stats_mean = pd.DataFrame(
    {
        'BRN': df[['BRN NODA 1', 'BRN NODA 2', 'BRN NODA 3', 'BRN NODA 4', 'BRN NODA 5']].mean(axis=1),
        'BRCG': df[['BRCG 1', 'BRCG 2', 'BRCG 3', 'BRCG 4', 'BRCG 5']].mean(axis=1),
        'DIF': df[['Dif 1', 'Dif 2', 'Dif 3', 'Dif 4', 'Dif 5']].mean(axis=1),
        'NAM': df[['NAM 1', 'NAM 2', 'NAM 3', 'NAM 4', 'NAM 5']].mean(axis=1),
        'CFN': df[['CoFR 1', 'CoFR 2', 'CoFR 3', 'CoFR 4', 'CoFR 5']].mean(axis=1),
        'FTT': df[['FFT 1', 'FFT 2', 'FFT 3', 'FFT 4', 'FFT 5']].mean(axis=1),
        'DAN': df[['DANet 1', 'DANet 2', 'DANet 3', 'DANet 4', 'DANet 5']].mean(axis=1),
        'MLP': df[['MLP NODA 1', 'MLP NODA 2', 'MLP NODA 3', 'MLP NODA 4', 'MLP NODA 5']].mean(axis=1),
        'RF': df[['RF 1', 'RF 2', 'RF 3', 'RF 4', 'RF 5']].mean(axis=1),
        'XGB': df[['XGB 1', 'XGB 2', 'XGB 3', 'XGB 4', 'XGB 5']].mean(axis=1),
        'GBT': df[['GBT 1', 'GBT 2', 'GBT 3', 'GBT 4', 'GBT 5']].mean(axis=1),
    }
)

In [ ]:
a = ['BRN', 'BRCG', 'DIF', 'NAM', 'CFN', 'FTT', 'DAN', 'MLP', 'RF', 'XGB', 'GBT']
b = ['BRN', 'BRCG', 'DIF', 'NAM', 'CFN', 'FTT', 'DAN', 'MLP', 'RF', 'XGB', 'GBT']

alg1 = []
alg2 = []
pval = []
for r in itertools.product(a, b):
    alg1 += [r[0]]
    alg2 += [r[1]]
    if r[0] != r[1]:
        d = np.around(df_stats_mean[r[0]] - df_stats_mean[r[1]], decimals=6)
        try:
            # res = wilcoxon(d, alternative='greater')
            res = mannwhitneyu(df_stats_mean[r[0]], df_stats_mean[r[1]])
        except:
            print(d, r)
        pval += [res.pvalue]
    else:
        pval += [np.nan]

results = pd.DataFrame({'alg1': alg1, 'alg2': alg2, 'pvalue_greater': pval})
results['alg1_mean_is_greater_than_algo2_significant'] = results['pvalue_greater'] <= 0.05
results[['alg1', 'alg2', 'alg1_mean_is_greater_than_algo2_significant']].set_index(['alg1', 'alg2']).unstack()

#### Produce overall P-Values vs BRN

In [ ]:
a = ['BRN', 'BRCG', 'DIF', 'NAM', 'CFN', 'FTT', 'DAN', 'MLP', 'RF', 'XGB', 'GBT']
b = ['BRN', 'BRCG', 'DIF', 'NAM', 'CFN', 'FTT', 'DAN', 'MLP', 'RF', 'XGB', 'GBT']

alg1 = []
alg2 = []
pval = []
for r in itertools.product(a, b):
    alg1 += [r[0]]
    alg2 += [r[1]]
    if r[0] != r[1]:
        d = np.around(df_stats_mean[r[0]] - df_stats_mean[r[1]], decimals=6)
        try:
            res = mannwhitneyu(df_stats_mean[r[0]], df_stats_mean[r[1]])
        except:
            print(d, r)
        pval += [res.pvalue]
    else:
        pval += [np.nan]

results = pd.DataFrame({'alg1': alg1, 'alg2': alg2, 'pvalue_greater': pval})
results[['alg1', 'alg2', 'pvalue_greater']].set_index(['alg1', 'alg2']).unstack().round(3)

# Explanation Evaluation

In [ ]:
exp_eval_df = pd.read_csv('path/to/results/explanation-eval-results.csv')

In [ ]:
exp_eval_df.head()